In [1]:
import pandas as pd

from bs4 import BeautifulSoup
import os
import requests
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import ndimage
from skimage.color import rgb2hsv
import skimage
from PIL import Image
import random
import cv2
import scipy.misc
from skimage import data, io
import pandas as pd

from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2gray

from sklearn.model_selection import train_test_split

In [2]:
pp = pd.read_csv('../data/picasso.csv')
hm = pd.read_csv('../data/henri_dic.csv')
from skimage import data, io

In [3]:
def RMS_CONTRAST(imagefile):
    print(imagefile)
    img = cv2.imread(imagefile)
    oneD_pixels = np.concatenate(img, axis=0)

    brightness = 0
    ## average brightness
    for info in oneD_pixels:
        brightness += (0.2126 * info[0]) + (0.7152 * info[1]) + (0.0722 * info[2])
    average_brightness = brightness / len(oneD_pixels)  ## calculate average
    average_brightness /= 255  ## normalizing to 0 and 1

    rms = 0
    for info in oneD_pixels:
        brightness = (0.2126 * info[0]) + (0.7152 * info[1]) + (0.0722 * info[2])
        brightness /= 255  ## normalizing to 0 and 1
        rms += pow((brightness - average_brightness), 2);  ## calculate squared
    rms /= len(oneD_pixels)  ## calculate mean squared
    rms = pow(rms, 0.5)
    return rms

def average_entropy(imagefile):
    print(imagefile)
    img = cv2.imread(imagefile)
    gray_img = rgb2gray(img)
    return entropy(gray_img, disk(5)).mean()

def calc_stats_mat(picture):
    result=[]
    width= picture.shape[1]
    height=picture.shape[0]
    rgb_img = picture
    hsv_img = rgb2hsv(rgb_img)
    hue_img = hsv_img[:, :, 0]
    saturation_img = hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    mean_hue = np.mean(hue_img, axis=(0,1))
    mean_saturation = np.mean(saturation_img, axis=(0,1))
    mean_brightness = np.mean(value_img)
    var_brightness = np.var(value_img)
    sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
    sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
    edge_image = np.hypot(sobel_x, sobel_y)
    sum_edge = np.sum(edge_image)/ edge_image.size
    result=[width,height,mean_hue,mean_saturation,mean_brightness,var_brightness,sum_edge,1]
    return result

def calc_stats_pic(picture):
    result=[]
    width= picture.shape[1]
    height=picture.shape[0]
    rgb_img = picture
    hsv_img = rgb2hsv(rgb_img)
    hue_img = hsv_img[:, :, 0]
    saturation_img = hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    mean_hue = np.mean(hue_img, axis=(0,1))
    mean_saturation = np.mean(saturation_img, axis=(0,1))
    mean_brightness = np.mean(value_img)
    var_brightness = np.var(value_img)
    sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
    sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
    edge_image = np.hypot(sobel_x, sobel_y)
    sum_edge = np.sum(edge_image)/ edge_image.size
    result=[width,height,mean_hue,mean_saturation,mean_brightness,var_brightness,sum_edge,0]
    return result

In [11]:
# your code here
display(os.path.join(os.pardir, '\data\henri-matisse'))
mat_dir = r'../data/henri-matisse'
pic_dir = r'../data/picasso'
db={}
for filename in os.listdir(mat_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        temp_path=os.path.join(mat_dir, filename)
        temp_picture=io.imread(temp_path)
        temp_result=calc_stats_mat(temp_picture)
        db[filename]=temp_result
    else:
        continue
for filename in os.listdir(pic_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        temp_path=os.path.join(pic_dir, filename)
        temp_picture=io.imread(temp_path)
        temp_result=calc_stats_pic(temp_picture)
        db[filename]=temp_result
    else:
        continue

'../\\data\\henri-matisse'

'..'

In [12]:
dataframe= pd.DataFrame.from_dict(db, orient='index',
columns=['width', 'height', 'mean_hue', 'mean_saturation','mean_brightness','variance_brightness','sum_edge','matis'])
dataframe["Picturesource"] = dataframe.index
dataframe

,width,height,mean_hue,mean_saturation,mean_brightness,variance_brightness,sum_edge,matis,Picturesource
paintings-orient_84_thmb.jpg,150,112,0.306358,0.318557,0.475946,0.060219,0.667396,1,paintings-orient_84_thmb.jpg
paintings-orient_179_thmb.jpg,135,206,0.144385,0.358490,0.514322,0.045731,0.426170,1,paintings-orient_179_thmb.jpg
paintings-thmbnls150-bj.jpg,150,179,0.214275,0.308174,0.774266,0.042401,0.572865,1,paintings-thmbnls150-bj.jpg
paintings-thmbnls150-zl.jpg,149,180,0.182714,0.411897,0.391973,0.041855,0.465982,1,paintings-thmbnls150-zl.jpg
paintings-orient_55_thmb.jpg,135,195,0.583862,0.404093,0.657891,0.031213,0.324745,1,paintings-orient_55_thmb.jpg
...,...,...,...,...,...,...,...,...,...
images-works-1718_s.jpg,160,200,0.440718,0.131216,0.614105,0.087034,0.644101,0,images-works-1718_s.jpg
images-works-1760_s.jpg,159,200,0.213146,0.705663,0.581691,0.048966,0.438055,0,images-works-1760_s.jpg
images-works-61_s.jpg,141,200,0.158872,0.559235,0.370588,0.079532,0.427156,0,images-works-61_s.jpg
images-works-128_s.jpg,151,200,0.457636,0.400556,0.688047,0.030584,0.370744,0,images-works-128_s.jpg


In [13]:
dataframe['RMS_contrast'] = dataframe.apply(lambda x: RMS_CONTRAST(os.path.join(mat_dir,x['Picturesource'])) if x['matis'] == 1 else RMS_CONTRAST(os.path.join(pic_dir,x['Picturesource'])), axis = 1)
dataframe['AVG_entropy'] = dataframe.apply(lambda x: average_entropy(os.path.join(mat_dir,x['Picturesource'])) if x['matis'] == 1 else average_entropy(os.path.join(pic_dir,x['Picturesource'])), axis = 1)

../data/henri-matisse/paintings-orient_84_thmb.jpg
../data/henri-matisse/paintings-orient_179_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-bj.jpg
../data/henri-matisse/paintings-thmbnls150-zl.jpg
../data/henri-matisse/paintings-orient_55_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-zn.jpg
../data/henri-matisse/paintings-thmbnls150-bh.jpg
../data/henri-matisse/paintings-thmbnls150-ad.jpg
../data/henri-matisse/paintings-orient_197_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-pntng43sm.jpg
../data/henri-matisse/paintings-thmbnls150-q_84.jpg
../data/henri-matisse/paintings-thmbnls150-q_85.jpg
../data/henri-matisse/paintings-thmbnls150-stilllifeb_sm.jpg
../data/henri-matisse/paintings-thmbnls150-ar.jpg
../data/henri-matisse/paintings-thmbnls150-bi.jpg
../data/henri-matisse/paintings-thmbnls150-zx.jpg
../data/henri-matisse/paintings-orient_118_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-hma322.jpg
../data/henri-matisse/paintings-thmbnls150-aa.jpg
../data/henri-ma

../data/picasso/images-works-1906_s.jpg
../data/picasso/images-works-2276_s.jpg
../data/picasso/images-works-1_s.jpg
../data/picasso/images-works-31_s.jpg
../data/picasso/images-works-89_s.jpg
../data/picasso/images-works-2702_s.jpg
../data/picasso/images-works-145_s.jpg
../data/picasso/images-works-2479_s.jpg
../data/picasso/images-works-102_s.jpg
../data/picasso/images-works-3010_s.jpg
../data/picasso/images-works-1513_s.jpg
../data/picasso/images-works-1941_s.jpg
../data/picasso/images-works-33_s.jpg
../data/picasso/images-works-3095_s.jpg
../data/picasso/images-works-17_s.jpg
../data/picasso/images-works-1878_s.jpg
../data/picasso/images-works-1885_s.jpg
../data/picasso/images-works-2832_s.jpg
../data/picasso/images-works-262_s.jpg
../data/picasso/images-works-1845_s.jpg
../data/picasso/images-works-4050_s.jpg
../data/picasso/images-works-1958_s.jpg
../data/picasso/images-works-1712_s.jpg
../data/picasso/images-works-3421_s.jpg
../data/picasso/images-works-56_s.jpg
../data/picasso/

../data/picasso/images-works-911_s.jpg
../data/picasso/images-works-4000_s.jpg
../data/picasso/images-works-2654_s.jpg
../data/picasso/images-works-3475_s.jpg
../data/picasso/images-works-1828_s.jpg
../data/picasso/images-works-2144_s.jpg
../data/picasso/images-works-2595_s.jpg
../data/picasso/images-works-1970_s.jpg
../data/picasso/images-works-2432_s.jpg
../data/picasso/images-works-1598_s.jpg
../data/picasso/images-works-1621_s.jpg
../data/picasso/images-works-1779_s.jpg
../data/picasso/images-works-174_s.jpg
../data/picasso/images-works-85_s.jpg
../data/picasso/images-works-1718_s.jpg
../data/picasso/images-works-1760_s.jpg
../data/picasso/images-works-61_s.jpg
../data/picasso/images-works-128_s.jpg
../data/picasso/images-works-2263_s.jpg
../data/henri-matisse/paintings-orient_84_thmb.jpg
../data/henri-matisse/paintings-orient_179_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-bj.jpg
../data/henri-matisse/paintings-thmbnls150-zl.jpg
../data/henri-matisse/paintings-orient_55_th

/usr/local/lib/python3.7/site-packages/skimage/filters/rank/generic.py:119: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  out_dtype)


../data/henri-matisse/paintings-orient_197_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-pntng43sm.jpg
../data/henri-matisse/paintings-thmbnls150-q_84.jpg
../data/henri-matisse/paintings-thmbnls150-q_85.jpg
../data/henri-matisse/paintings-thmbnls150-stilllifeb_sm.jpg
../data/henri-matisse/paintings-thmbnls150-ar.jpg
../data/henri-matisse/paintings-thmbnls150-bi.jpg
../data/henri-matisse/paintings-thmbnls150-zx.jpg
../data/henri-matisse/paintings-orient_118_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-hma322.jpg
../data/henri-matisse/paintings-thmbnls150-aa.jpg
../data/henri-matisse/paintings-thmbnls150-bz.jpg
../data/henri-matisse/paintings-orient_150_thmb.jpg
../data/henri-matisse/paintings-orient_127_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-ch.jpg
../data/henri-matisse/paintings-orient_99_thmb.jpg
../data/henri-matisse/paintings-thmbnls150-ab.jpg
../data/henri-matisse/paintings-thmbnls150-by.jpg
../data/henri-matisse/paintings-thmbnls150-cj.jpg
../data/henri-m

../data/picasso/images-works-33_s.jpg
../data/picasso/images-works-3095_s.jpg
../data/picasso/images-works-17_s.jpg
../data/picasso/images-works-1878_s.jpg
../data/picasso/images-works-1885_s.jpg
../data/picasso/images-works-2832_s.jpg
../data/picasso/images-works-262_s.jpg
../data/picasso/images-works-1845_s.jpg
../data/picasso/images-works-4050_s.jpg
../data/picasso/images-works-1958_s.jpg
../data/picasso/images-works-1712_s.jpg
../data/picasso/images-works-3421_s.jpg
../data/picasso/images-works-56_s.jpg
../data/picasso/images-works-2579_s.jpg
../data/picasso/images-works-3584_s.jpg
../data/picasso/images-works-3501_s.jpg
../data/picasso/images-works-122_s.jpg
../data/picasso/images-works-4070_s.jpg
../data/picasso/images-works-106_s.jpg
../data/picasso/images-works-1736_s.jpg
../data/picasso/images-works-1773_s.jpg
../data/picasso/images-works-183_s.jpg
../data/picasso/images-works-2741_s.jpg
../data/picasso/images-works-4008_s.jpg
../data/picasso/images-works-2442_s.jpg
../data/pi

../data/picasso/images-works-128_s.jpg
../data/picasso/images-works-2263_s.jpg


In [14]:
df = dataframe
df

,width,height,mean_hue,mean_saturation,mean_brightness,variance_brightness,sum_edge,matis,Picturesource,RMS_contrast,AVG_entropy
paintings-orient_84_thmb.jpg,150,112,0.306358,0.318557,0.475946,0.060219,0.667396,1,paintings-orient_84_thmb.jpg,0.207356,5.481119
paintings-orient_179_thmb.jpg,135,206,0.144385,0.358490,0.514322,0.045731,0.426170,1,paintings-orient_179_thmb.jpg,0.173101,5.042571
paintings-thmbnls150-bj.jpg,150,179,0.214275,0.308174,0.774266,0.042401,0.572865,1,paintings-thmbnls150-bj.jpg,0.204886,5.415232
paintings-thmbnls150-zl.jpg,149,180,0.182714,0.411897,0.391973,0.041855,0.465982,1,paintings-thmbnls150-zl.jpg,0.161756,4.767708
paintings-orient_55_thmb.jpg,135,195,0.583862,0.404093,0.657891,0.031213,0.324745,1,paintings-orient_55_thmb.jpg,0.176432,4.537113
...,...,...,...,...,...,...,...,...,...,...,...
images-works-1718_s.jpg,160,200,0.440718,0.131216,0.614105,0.087034,0.644101,0,images-works-1718_s.jpg,0.298298,5.365889
images-works-1760_s.jpg,159,200,0.213146,0.705663,0.581691,0.048966,0.438055,0,images-works-1760_s.jpg,0.142179,4.642326
images-works-61_s.jpg,141,200,0.158872,0.559235,0.370588,0.079532,0.427156,0,images-works-61_s.jpg,0.235527,4.616147
images-works-128_s.jpg,151,200,0.457636,0.400556,0.688047,0.030584,0.370744,0,images-works-128_s.jpg,0.197765,4.837914


In [15]:
dataframe.to_csv (r'../data/classifier.csv', index = False, header=True)

In [16]:
ents = ["width","height","mean_hue","mean_saturation","mean_brightness","variance_brightness","sum_edge","RMS_contrast","AVG_entropy"]
X = df[ents]
Y = df["matis"]

In [17]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [18]:
from sklearn import svm
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'xgboost'

# SVC

In [451]:
clf = svm.SVC()
clf.fit(x_train, y_train)

C:\Users\j3543\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [452]:
y_pred = clf.predict(x_train)
accuracy_score(y_train, y_pred)

0.9836065573770492

In [453]:
y_pred_test = clf.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.974025974025974

# DecisionTreeClassifier

In [454]:
clf1 = DecisionTreeClassifier(random_state=0)
clf1.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [455]:
y_pred1 = clf1.predict(x_train)
accuracy_score(y_train, y_pred1)

1.0

In [456]:
y_pred1 = clf1.predict(x_train)

In [457]:
y_pred_test = clf1.predict(x_test)
accuracy_score(y_test, y_pred_test)

1.0

# BernoulliNB

In [458]:
clf2 = BernoulliNB()
clf2.fit(x_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [459]:
y_pred2 = clf2.predict(x_train)
accuracy_score(y_train, y_pred2)

0.580327868852459

In [460]:
y_pred_test = clf2.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.6493506493506493

# KNeighborsClassifier

In [461]:
from sklearn.neighbors import KNeighborsClassifier
clf3 = KNeighborsClassifier(n_neighbors=4, p=2, leaf_size=60)
clf3.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=60, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [462]:
y_pred3 = clf3.predict(x_train)
accuracy_score(y_train, y_pred3)

0.9606557377049181

In [463]:
y_pred_test = clf3.predict(x_test)
accuracy_score(y_test, y_pred_test)

0.987012987012987

# RandomForestClassifier

In [464]:
clf4 = RandomForestClassifier(max_depth=14, random_state=10, n_estimators=20)
clf4.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=14, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [465]:
y_pred4 = clf4.predict(x_train)
accuracy_score(y_train, y_pred4)

1.0

In [466]:
y_pred_test = clf4.predict(x_test)
accuracy_score(y_test, y_pred_test)

1.0

# PCA

In [389]:
clf5 = PCA(n_components=2)
clf5.fit(x_train, y_train)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [390]:
y_pred5 = clf5.predict(x_train)
accuracy_score(y_train, y_pred5)

AttributeError: 'PCA' object has no attribute 'predict'

In [223]:
y_pred_test = clf5.predict(x_test)
accuracy_score(y_test, y_pred_test)

AttributeError: 'PCA' object has no attribute 'predict'